In [54]:
#module and extension import (Do not Change)
import pandas as pd
import numpy as np
import gspread
import re
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [55]:
#initialize required variables (Do Not Change)
scope = ['https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive']

#service account key (Do Not Change)
#google_account_key = 'UPVandLeads_Key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name ('/Volumes/KINGSTON/Etang LLV/Report/November 2022/UPVandLeads_Key.json', scope)

gc = gspread.authorize(credentials)

In [56]:
#Data process 1 (jumlah UPV CMS) change file name to 'UPVLeadsJumlah.xlsx'
dat1 = pd.read_excel('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Beda/UPVLeadsJumlah.xlsx')
dat1_inhouse = dat1[dat1['Agent Type'] == 'inhouse'].sort_values("Project").drop(labels=['Leads'], axis=1)
dat1_jv = dat1[dat1['Agent Type'] == 'inhouse_jv'].sort_values("Project")
dat1_ds = dat1[dat1['Agent Type'] == 'digital_sales'].sort_values("Project").drop(labels=['Leads'], axis=1)

In [57]:
#Data prcess 2 (Data CMS extract Email) Change file name to 'agentslist.xlsx'
agent= pd.read_excel('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Beda/agentslist.xlsx')
active= agent[agent['Status'] != 'Nonactive']
email = active.drop(columns= ['Phone', 'Type', 'Captain', 'Office', 'Project', 'Tanggal Pembuatan', 'URL Seller Page'])
datmail_inhouse= pd.merge(dat1_inhouse, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')
datmail_jv= pd.merge(dat1_jv, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')
datmail_ds= pd.merge(dat1_ds, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')

In [58]:
#Saving data 1 copy to each folder (Do Not Change)
today = date.today().strftime('%m-%d-%Y')
ext = '.xlsx'
comp = today + ext

datmail_inhouse.to_excel(r'/Volumes/KINGSTON/Etang LLV/Report/November 2022/Inhouse/Data-Inhouse-'+comp, index = False)
datmail_jv.to_excel(r'/Volumes/KINGSTON/Etang LLV/Report/November 2022/inhouseJV/Data-JV-'+comp, index=False)
datmail_ds.to_excel(r'/Volumes/KINGSTON/Etang LLV/Report/November 2022/DigitalSales/Data-DS-'+comp, index=False)

In [59]:
#Data process 3 (cleaning SAP Leads data) change file name to 'SAPbetul'
dat2 = pd.read_csv('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Beda/SAPbetul.csv') #ganti path
dat2.columns.str.match("Unnamed")
dat22 =dat2.loc[:,~dat2.columns.str.match("Unnamed")]
dat2_2nd = dat22.drop(columns=['Source - ReportingX', 'Phone', 'Year Month (Lead Submission Date)'])
#dat2_2nd = dat2_1st[dat2_1st['Status'] != 'Dropped'] #.drop_duplicates(subset= 'Account', keep= 'first')
dat2_2nd['Source1tf'] = dat2_2nd['Lead Source Name'].str.findall('ECAT.+') #Regex
dat2_2nd['source2tf'] = dat2_2nd['Source'].str.findall('.+ecat.+') #Regex
#dat2_2nd.to_excel(r'/Users/graceserenata23/Desktop/Etang LLV/UPV Leads Tracker App/test1.xlsx')
#'Lead Source Name', 'Source',

In [60]:
#filtering Ecatalog Only (Do Not Change)
dat2_2nd['source_t/f'] = dat2_2nd['Source1tf'].astype(str) + '-' + dat2_2nd['source2tf'].astype(str)
dat3 = dat2_2nd[dat2_2nd['source_t/f'] != '[]-[]']
dat3_2nd = dat3.drop(columns=['Source1tf', 'source2tf'])
dat3_2nd.head(5)
#dat3_2nd.to_excel(r'/Users/graceserenata23/Desktop/Etang LLV/UPV Leads Tracker App/saptest.xlsx')
#dat2_2nd.to_excel(r'/Users/graceserenata23/Desktop/Etang LLV/UPV Leads Tracker App/test1.xlsx')

,Lead,Lead ID,Lead Source Name,Lead Submission Date,Source - Reporting,Tag,utm_campaign,utm_content,Account,E-Mail,...,Tower/Cluster,ID of Successor Opportunity,Opportunity ID,All Leads,Accepted Leads,Reachable Leads,Follow Up Leads,Hot Prospect Leads,Won Leads,source_t/f
206,Anderson,404093,ECATALOG-NONDD-LUMINA AT THE OSTARA-WHATSAPPEC...,11/18/2022,Online,agentecatalog,Organic,Organic,Anderson,#,...,Not assigned,Not assigned,#,1,1.0,1.0,1.0,NaN,NaN,['ECATALOG-NONDD-LUMINA AT THE OSTARA-WHATSAPP...
213,anna sofia,401285,ECATALOG-RESIDENTIAL-BSD RESIDENTIAL-CHAVA AT ...,11/14/2022,Online,ecatalogOrganic,Organic,Organic,anna sofia,annasofia@gmail.com,...,Tanakayu Jiva,anna sofia,53227,1,1.0,1.0,1.0,NaN,NaN,['ECATALOG-RESIDENTIAL-BSD RESIDENTIAL-CHAVA A...
331,Apartement Klaska Residences - Eva,407102,ECATALOG-DD-CLUSTER-KLASKA RESIDENCE/13827,11/23/2022,Online,FBIGOrganic,ecatalog_Organic,link_tree_ig_klaskaresidence_projectpage,Eva,evaevo13@gmail.com,...,Tower Azure,Not assigned,#,1,1.0,1.0,1.0,NaN,NaN,['ECATALOG-DD-CLUSTER-KLASKA RESIDENCE/13827']-[]
405,Apartement Klaska Residences - Lenci,407095,ECATALOG-NONDD-AZURE TOWER AT KLASKA RESIDENCE...,11/23/2022,Online,SMLWebsite,Digital_2021,WebsiteSML_eCatalog_Button_Klaska_Reside,Lenci,#,...,Tower Azure,Not assigned,#,1,1.0,1.0,NaN,NaN,NaN,['ECATALOG-NONDD-AZURE TOWER AT KLASKA RESIDEN...
452,Apartement Klaska Residences - Neni,405609,ECATALOG-APARTMENT-KLASKA RESIDENCE-AZURE TOWE...,11/21/2022,Online,agentecatalog,Organic,Organic,Neni,#,...,Tower Azure,Apartement Klaska Residences - Neni,53356,1,1.0,1.0,1.0,NaN,NaN,['ECATALOG-APARTMENT-KLASKA RESIDENCE-AZURE TO...


In [61]:
#leads count per agent
dat3_2nd['Leads'] = dat3_2nd['E-Mail-Marketing'].map(dat3_2nd['E-Mail-Marketing'].value_counts())
dat3_email = dat3_2nd [['E-Mail-Marketing', 'Employee Responsible - Marketing', 'Leads']].drop_duplicates(subset='E-Mail-Marketing', keep='first')
#dat3_2nd.head(5)
#dat2count.to_excel(r'/Users/graceserenata23/Desktop/Etang LLV/UPV Leads Tracker App/testcount.xlsx', index= False)

In [62]:
#Creating Customer Journey
ney = dat3_2nd
journey = []

#making conditionals
ney2 = ney[['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads']]
ney['sum'] = ney2.count(axis=1)

for row in ney['sum']:
    if row == 1:
        journey.append('All Leads')
    elif row == 2:
        journey.append('Accepted Leads')
    elif row == 3:
        journey.append('Reachable Leads')
    elif row == 4:
        journey.append('Follow Up Leads')
    elif row == 5:
        journey.append('Hot Prospect Leads')
    elif row == 6:
        journey.append('WON Leads') 

    else:
        journey.append('NO JOURNEY')
        
ney['Journey'] = journey


In [63]:
#proses bentuk data SAP Leads
leadsacc1 = ney.drop(columns= ['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads', 'Leads', 'sum'])

leadsacc1['email-lower'] = leadsacc1['E-Mail-Marketing'].str.lower()
agentcode_in = datmail_inhouse[['Unique Code', 'Email']]
agentcode_ds = datmail_ds[['Unique Code', 'Email']]
agentcode_in['email-lower'] = agentcode_in['Email'].str.lower()
agentcode_ds['email-lower'] = agentcode_ds['Email'].str.lower()

leadsacc2 = pd.merge(leadsacc1, agentcode_in, how= 'left')
leadsacc3= pd.merge(leadsacc2, agentcode_ds, how= 'left')

#project V2 agent
leadsv1 = pd.read_csv('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Fix/projectdancode.csv')
leadsv2 = leadsv1[['Unique Code', 'Project V2']]
leadsv2['Project'] = leadsv2['Project V2']
sapmerge = pd.merge(leadsacc3, leadsv2, how= 'left')
#leadsv2.to_excel(r'/Users/graceserenata23/Desktop/Etang LLV/UPV Leads Tracker App/testfile/testproj.xlsx')

#BSDTeam
team1 = pd.read_csv('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Fix/BSDteam.csv')
team2 = team1[['Unique Code', 'BSD Team2']]
sapmerge2 = pd.merge(sapmerge, team2, how= 'left').drop(columns=['source_t/f', 'Project V2', 'email-lower']).rename(columns={'BSD Team2':'BSD Team'})
#sapmerge2.head(5)

#Lead Nature
Ln = pd.read_excel('/Volumes/KINGSTON/Etang LLV/Report/November 2022/Data Beda/enquiries.xlsx')
Ln1 = Ln[['Lead Nature', 'Type', 'Unique Code']].drop_duplicates(subset='Unique Code', keep='first')
sapmerge3 = pd.merge(sapmerge2, Ln1, how= 'left')

/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2221631050.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agentcode_in['email-lower'] = agentcode_in['Email'].str.lower()
/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2221631050.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agentcode_ds['email-lower'] = agentcode_ds['Email'].str.lower()
/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2221631050.py:16: SettingWithCopyWarning: 
A value is trying to

In [64]:
#merging procedure to count Leads (Inhouse)
dat3_email['email-lower'] = dat3_email['E-Mail-Marketing'].str.lower()
datmail_inhouse['email-lower'] = datmail_inhouse['Email'].str.lower()

merge_in= pd.merge(dat3_email, datmail_inhouse, how= 'outer')
merge_indrop = merge_in.drop(columns= ['E-Mail-Marketing', 'email-lower', 'Referral Link', 'Employee Responsible - Marketing']).dropna(subset= ['Unique Code'])
merge_indrop['UPV'] = merge_indrop['UPV']
merge_indrop['Leads'] = merge_indrop['Leads'].fillna(0)
merge_indrop2= merge_indrop[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']]

#changing project to project V2
leadsnoproj = leadsv2.drop(columns=['Project'])
merge_in2 = pd.merge(merge_indrop2, leadsnoproj, how='left')
#adding bsd team
merge_in3 = pd.merge(merge_in2, team2, how= 'left').drop(columns= ['Project']).rename(columns= {'BSD Team2':'BSD Team', 'Project V2':'Project', 'Nama_x':'Nama Agent'} )
merge_in3.to_excel(r'/Volumes/KINGSTON/Etang LLV/Report/November 2022/Inhouse/InhouseMerge.xlsx'+comp, index= False)

In [65]:
merge_in3.loc['Total'] = merge_in3.select_dtypes(pd.np.number).sum().fillna(0)

/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/3862144713.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  merge_in3.loc['Total'] = merge_in3.select_dtypes(pd.np.number).sum().fillna(0)


In [66]:
#JV data CMS only
datmail_jv2 = datmail_jv[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']].rename(columns={'Nama_x':'Nama Agent'})
#datmail_jv2.loc['Total'] = datmail_jv2.select_dtypes(pd.np.number).sum()

#finding leads on enquiries
enq2 = Ln[Ln['Is Duplicate'] != 'YES']
enq22 = enq2[enq2['Agent Type'] == 'inhouse_jv']

#merging leads to jv only
enq2['Lead'] = enq2['Agent Email'].map(enq2['Agent Email'].value_counts())
enq3 = enq2[['Unique Code','Lead']].drop_duplicates(subset='Unique Code', keep= 'first')
jv = pd.merge(datmail_jv2, enq3, how= 'left').fillna(0).drop(columns=['Leads'])
jv.loc['Total'] = jv.select_dtypes(pd.np.number).sum()


/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2937773260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enq2['Lead'] = enq2['Agent Email'].map(enq2['Agent Email'].value_counts())
/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2937773260.py:13: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  jv.loc['Total'] = jv.select_dtypes(pd.np.number).sum()


In [67]:
#merging procedure to count Leads (Digital Sales)
datmail_ds['email-lower'] = datmail_ds['Email'].str.lower()

merge_ds = pd.merge(dat3_email, datmail_ds, how= 'outer')
merge_dsdrop = merge_ds.drop(columns= ['E-Mail-Marketing', 'email-lower', 'Referral Link', 'Employee Responsible - Marketing']).dropna(subset= ['Unique Code'])
merge_dsdrop['UPV'] = merge_dsdrop['UPV']
merge_dsdrop['Leads'] = merge_dsdrop['Leads'].fillna(0)
merge_dsdrop2= merge_dsdrop[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']].rename(columns={'Nama_x':'Nama Agent'})
merge_dsdrop2.to_excel(r'/Volumes/KINGSTON/Etang LLV/Report/November 2022/DigitalSales/DSMerge.xlsx'+comp, index= False)

In [68]:
merge_dsdrop2.loc['Total'] = merge_dsdrop2.select_dtypes(pd.np.number).sum().fillna(0)

/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/434504865.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  merge_dsdrop2.loc['Total'] = merge_dsdrop2.select_dtypes(pd.np.number).sum().fillna(0)


In [69]:
#Not assigned Leads
dat3_3rd = dat3_2nd[dat3_2nd['Employee Responsible - Marketing'] == 'Not assigned']
Na = dat3_3rd[['Lead', 'Lead ID', 'Lead Source Name', 'Account', 'Mobile', 'Status', 'Employee Responsible - Marketing', 'Sales Unit', 'Project/Kawasan', 'Reason for Status', 'All Leads', 'Journey']]
Na['Total'] = Na.select_dtypes(pd.np.number).sum().fillna(0)

/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2669421085.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  Na['Total'] = Na.select_dtypes(pd.np.number).sum().fillna(0)
/var/folders/3q/57swpd011mz8jfmw2_lj13480000gn/T/ipykernel_41844/2669421085.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Na['Total'] = Na.select_dtypes(pd.np.number).sum().fillna(0)


In [70]:
#This is WorkSheet ID
Sheet_id = '1FdcvQTLrRfro-x6lQz9dq-MLrsgQnzwoTP7JSlyTJq4'
#This is Sheet Name
Sheet_inhouse = 'Inhouse'
Sheet_JV = 'Inhouse-JV'
Sheet_DS = 'Digital-Sales'
Sheet_NA = 'Not assigned'
Sheet_SAP = 'Data-SAP'
Sheet_CMS = 'Data-CMS'

#Upload Command
d2g.upload(merge_in3, Sheet_id, Sheet_inhouse, credentials= credentials, row_names= False)
d2g.upload(jv, Sheet_id, Sheet_JV, credentials= credentials, row_names= False)
d2g.upload(merge_dsdrop2, Sheet_id, Sheet_DS, credentials= credentials, row_names= False)
d2g.upload(Na, Sheet_id, Sheet_NA, credentials= credentials, row_names= False)
d2g.upload(sapmerge3, Sheet_id, Sheet_SAP, credentials= credentials, row_names= False)
d2g.upload(enq22, Sheet_id, Sheet_CMS, credentials= credentials, row_names= False)

<Worksheet 'Data-CMS' id:648469926>